***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [2]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-7c98  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [3]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [4]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import numpy as np
import pandas as pd
from google.cloud import storage
from datetime import datetime as dt

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 13 19:02 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [6]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [7]:
spark

In [8]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'david-ir-bucket' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if "parquet" in b.name:
        paths.append(full_path + b.name)

60


***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [9]:
parquetFile = spark.read.parquet(*paths)
print(paths)

['gs://david-ir-bucket/multistream10_preprocessed.parquet', 'gs://david-ir-bucket/multistream11_part2_preprocessed.parquet', 'gs://david-ir-bucket/multistream11_preprocessed.parquet', 'gs://david-ir-bucket/multistream12_part2_preprocessed.parquet', 'gs://david-ir-bucket/multistream12_preprocessed.parquet', 'gs://david-ir-bucket/multistream13_part2_preprocessed.parquet', 'gs://david-ir-bucket/multistream13_preprocessed.parquet', 'gs://david-ir-bucket/multistream14_part2_preprocessed.parquet', 'gs://david-ir-bucket/multistream14_preprocessed.parquet', 'gs://david-ir-bucket/multistream15_part2_preprocessed.parquet', 'gs://david-ir-bucket/multistream15_part3_preprocessed.parquet', 'gs://david-ir-bucket/multistream15_preprocessed.parquet', 'gs://david-ir-bucket/multistream16_part2_preprocessed.parquet', 'gs://david-ir-bucket/multistream16_part3_preprocessed.parquet', 'gs://david-ir-bucket/multistream16_preprocessed.parquet', 'gs://david-ir-bucket/multistream17_part2_preprocessed.parquet', '

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [10]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [11]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [12]:
from inverted_index_gcp import InvertedIndex

In [13]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [14]:
english_stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = set(english_stopwords+corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)    
def tokening(text):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    l = [x for x in tokens if x not in all_stopwords]
    return l
def make_index(rdd:pyspark.RDD, index_name:str, num_of_buckets:int=124,stem=False, minmax=False,remove_top=0):
    t = dt.now()
    stemdict = defaultdict()
    def word_count(text, id):
        
        ''' Count the frequency of each word in `text` (tf) that is not included in
        `all_stopwords` and return entries that will go into our posting lists.
        Parameters:
        -----------
          text: str
            Text of one document
          id: int
            Document id
        Returns:
        --------
          List of tuples
            A list of (token, (doc_id, tf)) pairs
            for example: [("Anarchism", (12, 5)), ...]
        '''
#         stemmer = PorterStemmer()
        
        tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
#         if stem:
#           stemmer = PorterStemmer()
#           ret = []
#           for x in tokens:
#             if x in stemdict:
#               ret.append(x)
#             else:
#               stemmed = stemmer.stem(x)
#               stemdict[x]=stemmed
#               ret.append(stemmed)
#           tokens = ret
        return [(x[0], (id, x[1])) for x in Counter([x for x in tokens if x not in all_stopwords]).items()]

    word_counts = rdd.flatMap(lambda x: word_count(x[0], x[1]))
    print("finished word countes: ", dt.now() - t)
    t = dt.now()

    def reduce_word_counts(unsorted_pl):
        ''' Returns a sorted posting list by wiki_id.
        Parameters:
        -----------
          unsorted_pl: list of tuples
            A list of (wiki_id, tf) tuples
        Returns:
        --------
          list of tuples
            A sorted posting list.
        '''
        return sorted(unsorted_pl, key=lambda x: x[0])

    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    print("finished reduce : ", dt.now() - t)
    t = dt.now()
    
#     top_n = [0 for x in range(remove_top)]

#     def find_max(x):
#         if any([x > y for y in top_n]):
#             top_n[top_n.index(min(top_n))] = x
#     if remove_top:
#         _ = postings.mapValues(lambda x: find_max(len(x))).collect()

    top_n = [1,2]
    def my_sum(l):
        s = 0
        for x in l:
            s+=x
        return s
    postings_filtered = postings.filter(lambda x:x)# len(x[1]) not in top_n or my_sum([y[1] for y in x[1]]) > 50)

    def calculate_df(postings):
        ''' Takes a posting list RDD and calculate the df for each token.
        Parameters:
        -----------
          postings: RDD
            An RDD where each element is a (token, posting_list) pair.
        Returns:
        --------
          RDD
            An RDD where each element is a (token, df) pair.
        '''
        # YOUR CODE HERE
        return postings.map(lambda x: (x[0], len(x[1])))

    w2df = calculate_df(postings_filtered)

    w2df_dict = w2df.collectAsMap()

    NUM_BUCKETS = num_of_buckets
    print("finished filter+calcdf : ", dt.now() - t)
    t = dt.now()

    # def _hash(s):
    #     return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()
    #
    def token2bucket_id(token):
        return int(_hash(token),16) % NUM_BUCKETS

    def partition_postings_and_write(postings):
        ''' A function that partitions the posting lists into buckets, writes out
        all posting lists in a bucket to disk, and returns the posting locations for
        each bucket. Partitioning should be done through the use of `token2bucket`
        above. Writing to disk should use the function  `write_a_posting_list`, a
        static method implemented in inverted_index_colab.py under the InvertedIndex
        class.
        Parameters:
        -----------
          postings: RDD
            An RDD where each item is a (w, posting_list) pair.
        Returns:
        --------
          RDD
            An RDD where each item is a posting locations dictionary for a bucket. The
            posting locations maintain a list for each word of file locations and
            offsets its posting list was written to. See `write_a_posting_list` for
            more details.
        '''
        # YOUR CODE HERE
#         postings = postings.filter(lambda x: x)
        print("hi")
        return postings.map(lambda x: (token2bucket_id(x[0]), [x])).reduceByKey(lambda x, y: x + y).map(
            lambda x: (InvertedIndex.write_a_posting_list(x,"david-ir-bucket")))

    posting_locs_list = partition_postings_and_write(postings_filtered).collect()
    print("finished  collect : ", dt.now() - t)
    t = dt.now()
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
      if not blob.name.endswith("pickle"):
        continue
      with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
          super_posting_locs[k].extend(v)
    print("finished  blobing : ", dt.now() - t)
    t = dt.now()

    # Create inverted index instance
    inverted = InvertedIndex(name=index_name)
    # Adding the posting locations dictionary to the inverted index
    inverted.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    inverted.df = w2df_dict
    # write the global stats out
    inverted.write_index(".", 'index')
    print("finished finished" , dt.now() - t)



In [ ]:
# Create inverted index instance
inverted = InvertedIndex(name=index_name)
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index(".", 'index')
print("why are you runnin")

In [ ]:
#here we make the meta dict for retrieving the title for each doc id and the size of the body for tf-idf and bm25 calculations.
meta_dict = parquetFile.select("id", "title", "text", "anchor_text").rdd
meta_dict = meta_dict.map(lambda x: (x[0], x[1], tokening(x[1]), tokening(x[2]), x[3]))
meta_dict = meta_dict.map(lambda x: (x[0], (x[1], len(x[2]), len(x[3]), len(x[4])))).collectAsMap()

In [ ]:
with open('meta_dict.pkl', 'wb') as handle:
    pickle.dump(id_title_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [41]:
#index making for the title
doc_title_pairs = parquetFile.select("title", "id").rdd
make_index(doc_title_pairs, "title_index", 124)

finished word countes:  0:00:00.000233
finished reduce :  0:00:00.013322


finished filter+calcdf :  0:00:23.089949
hi


finished  collect :  0:00:10.094514
finished  blobing :  0:00:12.959482
values
listed
nped
16002967
finished finished 0:00:03.657185


In [44]:
#transfering the index from local to the bucket and then from postings_gcp folder to index name folder(change for each index we needed to transfer)
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst
index_src = f'gs://{bucket_name}/postings_gcp' 
index_dst = f'gs://{bucket_name}/anchor_index'
!gsutil mv $index_src $index_dst  

Copying file://index.pkl [Content-Type=application/octet-stream]...
\ [1 files][ 94.6 MiB/ 94.6 MiB]                                                
Operation completed over 1 objects/94.6 MiB.                                     
Copying gs://david-ir-bucket/postings_gcp/0_000.bin [Content-Type=application/octet-stream]...
Removing gs://david-ir-bucket/postings_gcp/0_000.bin...                         
Copying gs://david-ir-bucket/postings_gcp/0_001.bin [Content-Type=application/octet-stream]...
Removing gs://david-ir-bucket/postings_gcp/0_001.bin...                         
Copying gs://david-ir-bucket/postings_gcp/0_002.bin [Content-Type=application/octet-stream]...
Removing gs://david-ir-bucket/postings_gcp/0_002.bin...                         
Copying gs://david-ir-bucket/postings_gcp/0_003.bin [Content-Type=application/octet-stream]...
Removing gs://david-ir-bucket/postings_gcp/0_003.bin...                         

==> NOTE: You are performing a sequence of gsutil operations tha

In [ ]:
#index making for the body text
doc_text_pairs = parquetFile.select("text", "id").rdd
make_index(doc_text_pairs, "body_index", 124)

finished word countes:  0:00:00.004984
finished reduce :  0:00:00.063073


finished filter+calcdf :  0:17:57.186266
hi


finished  collect :  0:09:58.529625
finished  blobing :  0:02:40.988506


In [ ]:
#try to stem the index
# stemmer = PorterStemmer()
doc_text_pairs = parquetFile.select("id","text").rdd
doc_text_pairs = doc_text_pairs.map(lambda x: tokening(x[1]))
# doc_text_pairs = doc_text_pairs.map(lambda x: (x[0], [stemmer.stem(y) for y in x[1]]))
# doc_text_pairs = doc_text_pairs.map(lambda x: " ".join(x[1]))
# doc_text_pairs = doc_text_pairs.map(lambda x: (x[0], merge_corpus(x[1])))
final_corpus = doc_text_pairs.collect()
print(len(final_corpus))


In [136]:
#figuring out the myread function for the fast info retrival
def myread(index,word,debug=0):
    f_name, offset = index.posting_locs[word][0][0],index.posting_locs[word][0][1]
    if debug: 
        print(f_name,offset)
    with open("body_index"+"/"+f_name, 'rb') as file:
        if debug:
            print(len(file.read()))
        file.seek(0)
        file.seek(offset)
        n_bytes = index.df[word]*6
        mylist = []
        for itr in range(n_bytes):
            tup = file.read(6)
            for i in range(6):
                print(len(tup))
                mylist.append((tup[0]*256**3 + tup[1]*256**2 + tup[2]*256 + tup[3],tup[4]*256 + tup[5]))
    return mylist

SyntaxError: invalid syntax (2872106411.py, line 15)

In [43]:
#index making for the anchor text
doc_anchor_pairs = parquetFile.select("id","anchor_text").rdd
doc_anchor_pairs = doc_anchor_pairs.flatMap(lambda x: x[1])
doc_anchor_pairs = doc_anchor_pairs.reduceByKey(lambda x,y: y if x in y else x if y in x else f"{x} {y}")
doc_anchor_pairs= doc_anchor_pairs.map(lambda x:(x[1],x[0]))
make_index(doc_anchor_pairs, "anchor_index", 124,stem=True)

finished word countes:  0:00:00.000025
finished reduce :  0:00:00.015040


finished filter+calcdf :  0:04:37.123766
hi


finished  collect :  0:00:32.008276
finished  blobing :  0:00:15.881102
values
listed
nped
157403886
finished finished 0:00:04.541061


# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
# Put your `generate_graph` function here//used the pr file we had in assignment 3

In [ ]:
t_start = time()
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

page views:

In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)
# read in the counter
with open(pv_clean, 'rb') as f:
  wid2pv = pickle.loads(f.read())